In [1]:
import mysql.connector
from dotenv import load_dotenv
from decouple import config 
import os

In [5]:
load_dotenv()

connection = mysql.connector.connect(
    host= os.getenv("host"),
    user= os.getenv("user"),
    password= os.getenv("password"),
    database="music_trends"
)

cursor = connection.cursor()

In [6]:
music_trends = """CREATE DATABASE "music_trends";"""